In [1]:
# Install necessary libraries
%pip install librosa scikit-learn

# Import libraries
import librosa
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
from IPython.display import display
from ipywidgets import FileUpload

# Define the full note mapping with microtones (adjust frequencies as needed)
note_mapping = {
    'sa': (240, 260),
    're': (260, 290),
    're#': (290, 310),  # Microtone
    'ga': (310, 330),
    'ma': (330, 370),
    'ma#': (370, 390),  # Microtone
    'pa': (390, 420),
    'dha': (420, 470),
    'ni': (470, 520),
    'ni#': (520, 540)   # Microtone
}

# Map frequency to note
def frequency_to_note(freq):
    for note, (low, high) in note_mapping.items():
        if low <= freq <= high:
            return note
    return None

# Generate features for training
def extract_features(audio_path, sr=22050, hop_length=512):
    try:
        y, _ = librosa.load(audio_path, sr=sr)
        stft = librosa.stft(y, n_fft=2048, hop_length=hop_length)
        pitches, magnitudes = librosa.piptrack(S=np.abs(stft), sr=sr)
        features = []

        for i in range(pitches.shape[1]):
            pitch_slice = pitches[:, i]
            mag_slice = magnitudes[:, i]
            if mag_slice.any():
                freq = pitch_slice[np.argmax(mag_slice)]
                if freq > 0:
                    features.append(freq)
        return features
    except Exception as e:
        print(f"Error extracting features: {e}")
        return []

# Train a machine learning model
def train_model():
    # Labeled data: Replace with your own dataset of frequencies and corresponding notes
    frequencies = [250, 270, 300, 320, 350, 380, 400, 450, 500, 530]
    labels = ['sa', 're', 're#', 'ga', 'ma', 'ma#', 'pa', 'dha', 'ni', 'ni#']

    # Convert to numpy arrays for training
    X = np.array(frequencies).reshape(-1, 1)
    y = np.array(labels)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train the Random Forest Classifier
    model = RandomForestClassifier(n_estimators=200, random_state=42)
    model.fit(X_train, y_train)

    # Evaluate the model
    predictions = model.predict(X_test)
    print("\nClassification Report:\n", classification_report(y_test, predictions))

    # Save the model
    with open("note_classifier.pkl", "wb") as f:
        pickle.dump(model, f)

    return model

# Predict notes in an audio file
def detect_notes_with_model(audio_path, model_path="note_classifier.pkl", sr=22050, hop_length=512):
    try:
        # Load the pre-trained model
        with open(model_path, "rb") as f:
            model = pickle.load(f)

        # Extract features from the audio file
        features = extract_features(audio_path, sr=sr, hop_length=hop_length)
        if not features:
            print("No features extracted. Please check the audio file.")
            return

        # Predict notes for each frequency
        predicted_notes = model.predict(np.array(features).reshape(-1, 1))

        # Time per frame (hop_length / sample_rate)
        time_per_frame = hop_length / sr

        # Display the detected notes with proper durations
        print("\nDetected Notes with Durations:")
        previous_note = None
        note_start_time = 0
        time_elapsed = 0
        detected_notes = []

        for i, note in enumerate(predicted_notes):
            current_time = i * time_per_frame
            if note != previous_note:
                if previous_note is not None:
                    duration = current_time - note_start_time
                    detected_notes.append((previous_note, note_start_time, duration))
                previous_note = note
                note_start_time = current_time

        # Add the last note
        if previous_note is not None:
            duration = len(predicted_notes) * time_per_frame - note_start_time
            detected_notes.append((previous_note, note_start_time, duration))

        # Print detected notes
        for note, start_time, duration in detected_notes:
            print(f"Note: {note}, Start Time: {start_time:.2f} seconds, Duration: {duration:.2f} seconds")

    except Exception as e:
        print(f"Error processing file: {e}")

# Train the model (run this once)
print("Training the model...")
train_model()

# Upload an audio file for detection
upload_widget = FileUpload(accept=".wav", multiple=False)
display(upload_widget)

print("\nUpload an audio file using the widget above and run the following cell to process it.")


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 260.1/260.1 kB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 11.1/11.1 MB 5.0 MB/s eta 0:00:00
     ---------------------------------------- 12.9/12.9 MB 4.4 MB/s eta 0:00:00
     ---------------------------------------- 43.9/43.9 MB 5.0 MB/s eta 0:00:00
     -------------------------------------- 301.8/301.8 kB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 2.7/2.7 MB 5.4 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 5.0 MB/s eta 0:00:00
     ---------------------------------------- 64.6/64.6 kB 3.4 MB/s eta 0:00:00
     -------------------------------------- 166.7/166.7 kB 5.0 MB/s eta 0:00:00
     ---------------------------------------- 74.9/74.9 kB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 28.1/28.1 MB 5.1 MB/s eta 0:00:00
     ---------------------------------------- 15.9/15.9 MB 5.4 MB/s eta 0:00:00
     ---------------------------------------

ModuleNotFoundError: No module named 'ipywidgets'